In [1]:
#stock price exploration with sparksql

In [4]:
#initialize spark context
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [6]:
#load data into Spark
tesla_file ='TSLA.csv'
tesla_rdd = sc.textFile(tesla_file)
tesla_rdd.take(5)

['Date,Open,High,Low,Close,AdjClose,Volume',
 '2019-07-15,248.000000,254.419998,244.860001,253.500000,253.500000,11000100',
 '2019-07-16,249.300003,253.529999,247.929993,252.380005,252.380005,8149000',
 '2019-07-17,255.669998,258.309998,253.350006,254.860001,254.860001,9764700',
 '2019-07-18,255.050003,255.750000,251.889999,253.539993,253.539993,4764500']

In [9]:
csv_rdd = tesla_rdd.map(lambda row:row.split(","))
csv_rdd.collect()

[['Date', 'Open', 'High', 'Low', 'Close', 'AdjClose', 'Volume'],
 ['2019-07-15',
  '248.000000',
  '254.419998',
  '244.860001',
  '253.500000',
  '253.500000',
  '11000100'],
 ['2019-07-16',
  '249.300003',
  '253.529999',
  '247.929993',
  '252.380005',
  '252.380005',
  '8149000'],
 ['2019-07-17',
  '255.669998',
  '258.309998',
  '253.350006',
  '254.860001',
  '254.860001',
  '9764700'],
 ['2019-07-18',
  '255.050003',
  '255.750000',
  '251.889999',
  '253.539993',
  '253.539993',
  '4764500'],
 ['2019-07-19',
  '255.690002',
  '259.959991',
  '254.619995',
  '258.179993',
  '258.179993',
  '7048400'],
 ['2019-07-22',
  '258.750000',
  '262.149994',
  '254.190002',
  '255.679993',
  '255.679993',
  '6842400'],
 ['2019-07-23',
  '256.709991',
  '260.480011',
  '254.500000',
  '260.170013',
  '260.170013',
  '5023100'],
 ['2019-07-24',
  '259.170013',
  '266.070007',
  '258.160004',
  '264.880005',
  '264.880005',
  '11072800'],
 ['2019-07-25',
  '233.500000',
  '234.500000',
  '22

In [12]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [21]:
tesla_df = csv_rdd.toDF(['date','open','high','low','close','adjclose','volume'])

In [15]:
amazon_file = 'AMZN.csv'
amazon_df = sqlContext.read.load(amazon_file,
                                format = 'com.databricks.spark.csv',
                                header = 'true',
                                inferSchema='true')

In [16]:
amazon_df.take(2)

[Row(Date='2019-07-15', Open=2021.400024, High=2022.900024, Low=2001.550049, Close=2020.98999, AdjClose=2020.98999, Volume=2981300),
 Row(Date='2019-07-16', Open=2010.579956, High=2026.319946, Low=2001.219971, Close=2009.900024, AdjClose=2009.900024, Volume=2618200)]

In [17]:
amazon_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- AdjClose: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [18]:
amazon_df.count()

253

In [20]:
import pandas
amazon_df.toPandas().head(10)

,Date,Open,High,Low,Close,AdjClose,Volume
0,2019-07-15,2021.400024,2022.900024,2001.550049,2020.989990,2020.989990,2981300
1,2019-07-16,2010.579956,2026.319946,2001.219971,2009.900024,2009.900024,2618200
2,2019-07-17,2007.050049,2012.000000,1992.030029,1992.030029,1992.030029,2558800
3,2019-07-18,1980.010010,1987.500000,1951.550049,1977.900024,1977.900024,3504300
4,2019-07-19,1991.209961,1996.000000,1962.229980,1964.520020,1964.520020,3185600
5,2019-07-22,1971.140015,1989.000000,1958.260010,1985.630005,1985.630005,2900000
6,2019-07-23,1995.989990,1997.790039,1973.130005,1994.489990,1994.489990,2703500
7,2019-07-24,1969.300049,2001.300049,1965.869995,2000.810059,2000.810059,2631300
8,2019-07-25,2001.000000,2001.199951,1972.719971,1973.819946,1973.819946,4136500
9,2019-07-26,1942.000000,1950.900024,1924.510010,1943.050049,1943.050049,4927100


In [22]:
tesla_file = 'TSLA.csv'
tesla_df = sqlContext.read.load(amazon_file,
                                format = 'com.databricks.spark.csv',
                                header = 'true',
                                inferSchema='true')

In [23]:
google_file = 'GOOG.csv'
google_df = sqlContext.read.load(amazon_file,
                                format = 'com.databricks.spark.csv',
                                header = 'true',
                                inferSchema='true')

In [26]:
from pyspark.sql.functions import year, month
google_df.select(year('Date').alias('yr'),'Close').groupby('yr').avg('Close').sort('yr').show()

+----+------------------+
|  yr|        avg(Close)|
+----+------------------+
|2019|1800.6161329411755|
|2020| 2236.414478798507|
+----+------------------+



In [32]:
amazon_df.select(year('Date').alias('Year'),month('Date').alias('Month'),'Low') \
.groupby('Year','Month').avg('Low').sort('Year','Month').show()

+----+-----+------------------+
|Year|Month|          avg(Low)|
+----+-----+------------------+
|2019|    7|1948.1946176153847|
|2019|    8|1778.2340920454546|
|2019|    9|     1784.68599235|
|2019|   10|1736.6047840869564|
|2019|   11|1764.3005004499998|
|2019|   12|1773.8466622380954|
|2020|    1| 1870.354288809524|
|2020|    2|2045.0563194210524|
|2020|    3|1825.1590964090908|
|2020|    4|2185.9347679523808|
|2020|    5|     2364.48549805|
|2020|    6| 2579.099553909091|
|2020|    7|2976.1799859999996|
+----+-----+------------------+



In [33]:
amazon_df.registerTempTable('amazon_stocks')

In [34]:
google_df.registerTempTable('google_stocks')

In [35]:
tesla_df.registerTempTable('tesla_stocks')

In [37]:
sqlContext.sql('SELECT * FROM amazon_stocks').show(5)

+----------+-----------+-----------+-----------+-----------+-----------+-------+
|      Date|       Open|       High|        Low|      Close|   AdjClose| Volume|
+----------+-----------+-----------+-----------+-----------+-----------+-------+
|2019-07-15|2021.400024|2022.900024|2001.550049| 2020.98999| 2020.98999|2981300|
|2019-07-16|2010.579956|2026.319946|2001.219971|2009.900024|2009.900024|2618200|
|2019-07-17|2007.050049|     2012.0|1992.030029|1992.030029|1992.030029|2558800|
|2019-07-18| 1980.01001|     1987.5|1951.550049|1977.900024|1977.900024|3504300|
|2019-07-19|1991.209961|     1996.0| 1962.22998| 1964.52002| 1964.52002|3185600|
+----------+-----------+-----------+-----------+-----------+-----------+-------+
only showing top 5 rows



In [39]:
sqlContext.sql('SELECT year(Date) as year, month(Date) as month, avg(Close) from amazon_stocks group by year, month').show()

+----+-----+------------------+
|year|month|        avg(Close)|
+----+-----+------------------+
|2019|   10|1752.3317498695653|
|2020|    6|      2613.5454545|
|2020|    3|1872.3104358636365|
|2019|    8|1793.6027220909093|
|2020|    4|2228.7052408571426|
|2020|    1|1884.2376128571425|
|2019|    9|     1799.12099615|
|2019|   12|1785.7728446190476|
|2020|    7| 3053.100016222222|
|2020|    2|2066.1752672631574|
|2019|    7|1964.6846265384618|
|2019|   11|      1774.2939941|
|2020|    5|2394.1840209499996|
+----+-----+------------------+



In [40]:
sqlContext.sql('SELECT Date, Open, Close, abs(Close - Open) as dif FROM google_stocks WHERE abs(Close - Open) > 4').show()

+----------+-----------+-----------+------------------+
|      Date|       Open|      Close|               dif|
+----------+-----------+-----------+------------------+
|2019-07-17|2007.050049|1992.030029|15.020019999999931|
|2019-07-19|1991.209961| 1964.52002| 26.68994100000009|
|2019-07-22|1971.140015|1985.630005|14.489990000000034|
|2019-07-24|1969.300049|2000.810059|31.510009999999966|
|2019-07-25|     2001.0|1973.819946|27.180053999999927|
|2019-07-29|     1930.0|1912.449951|17.550048999999944|
|2019-07-30|1891.119995|1898.530029| 7.410033999999996|
|2019-07-31|1898.109985|1866.780029| 31.32995600000004|
|2019-08-01|1871.719971|1855.319946|16.400024999999914|
|2019-08-02|1845.069946| 1823.23999| 21.82995600000004|
|2019-08-05|1770.219971|1765.130005| 5.089966000000004|
|2019-08-06| 1792.22998|1787.829956|  4.40002400000003|
|2019-08-07| 1773.98999|1793.400024|19.410033999999996|
|2019-08-08|     1806.0|1832.890015| 26.89001499999995|
|2019-08-09|1828.949951|1807.579956|21.369995000

In [43]:
sqlContext.sql("SELECT year(Date) AS year, \
                MAX(AdjClose), MIN(AdjClose) \
                FROM tesla_stocks \
                GROUP BY year").show()

+----+-------------+-------------+
|year|max(AdjClose)|min(AdjClose)|
+----+-------------+-------------+
|2019|   2020.98999|   1705.51001|
|2020|       3200.0|  1676.609985|
+----+-------------+-------------+



In [52]:
joinclose_df = sqlContext.sql("SELECT tesla_stocks.Date, \
               tesla_stocks.Close AS teslaClose, \
               amazon_stocks.Close AS amazonClose, \
               google_stocks.Close AS googleClose \
               FROM tesla_stocks \
               JOIN google_stocks ON tesla_stocks.Date = google_stocks.Date \
               JOIN amazon_stocks ON amazon_stocks.Date = tesla_stocks.Date")

In [53]:
joinclose_df.show()

+----------+-----------+-----------+-----------+
|      Date| teslaClose|amazonClose|googleClose|
+----------+-----------+-----------+-----------+
|2019-07-15| 2020.98999| 2020.98999| 2020.98999|
|2019-07-16|2009.900024|2009.900024|2009.900024|
|2019-07-17|1992.030029|1992.030029|1992.030029|
|2019-07-18|1977.900024|1977.900024|1977.900024|
|2019-07-19| 1964.52002| 1964.52002| 1964.52002|
|2019-07-22|1985.630005|1985.630005|1985.630005|
|2019-07-23| 1994.48999| 1994.48999| 1994.48999|
|2019-07-24|2000.810059|2000.810059|2000.810059|
|2019-07-25|1973.819946|1973.819946|1973.819946|
|2019-07-26|1943.050049|1943.050049|1943.050049|
|2019-07-29|1912.449951|1912.449951|1912.449951|
|2019-07-30|1898.530029|1898.530029|1898.530029|
|2019-07-31|1866.780029|1866.780029|1866.780029|
|2019-08-01|1855.319946|1855.319946|1855.319946|
|2019-08-02| 1823.23999| 1823.23999| 1823.23999|
|2019-08-05|1765.130005|1765.130005|1765.130005|
|2019-08-06|1787.829956|1787.829956|1787.829956|
|2019-08-07|1793.400

In [58]:
joinclose_df1 = sqlContext.sql("SELECT  YEAR(tesla_stocks.Date) AS YEAR, \
               MONTH(tesla_stocks.DATE) AS MONTH, \
               AVG(tesla_stocks.Close) AS AvgTeslaClose, \
               AVG(amazon_stocks.Close) AS AvgAmazonClose, \
               AVG(google_stocks.Close) AS AvgGoogleClose \
               FROM tesla_stocks \
               JOIN google_stocks ON tesla_stocks.Date = google_stocks.Date \
               JOIN amazon_stocks ON amazon_stocks.Date = tesla_stocks.Date \
               GROUP BY YEAR, MONTH \
               ORDER BY YEAR, MONTH")

In [59]:
joinclose_df1.show()

+----+-----+------------------+------------------+------------------+
|YEAR|MONTH|     AvgTeslaClose|    AvgAmazonClose|    AvgGoogleClose|
+----+-----+------------------+------------------+------------------+
|2019|    7|1964.6846265384618|1964.6846265384618|1964.6846265384618|
|2019|    8|1793.6027220909093|1793.6027220909093|1793.6027220909093|
|2019|    9|     1799.12099615|     1799.12099615|     1799.12099615|
|2019|   10|1752.3317498695653|1752.3317498695653|1752.3317498695653|
|2019|   11|      1774.2939941|      1774.2939941|      1774.2939941|
|2019|   12|1785.7728446190476|1785.7728446190476|1785.7728446190476|
|2020|    1|1884.2376128571425|1884.2376128571425|1884.2376128571425|
|2020|    2|2066.1752672631574|2066.1752672631574|2066.1752672631574|
|2020|    3|1872.3104358636365|1872.3104358636365|1872.3104358636365|
|2020|    4|2228.7052408571426|2228.7052408571426|2228.7052408571426|
|2020|    5|2394.1840209499996|2394.1840209499996|2394.1840209499996|
|2020|    6|      26

In [60]:
#Save Spark Dataframes in Parquet format
joinclose_df.write.format("parquet").save("joinStock.parquet")
joinclose_df1.write.format("parquet").save("joinStock1.parquet")

In [61]:
final_df = sqlContext.read.parquet("joinStock.parquet")
final_df1 = sqlContext.read.parquet("joinStock1.parquet")

In [62]:
final_df.show()

+----------+-----------+-----------+-----------+
|      Date| teslaClose|amazonClose|googleClose|
+----------+-----------+-----------+-----------+
|2019-07-15| 2020.98999| 2020.98999| 2020.98999|
|2019-07-16|2009.900024|2009.900024|2009.900024|
|2019-07-17|1992.030029|1992.030029|1992.030029|
|2019-07-18|1977.900024|1977.900024|1977.900024|
|2019-07-19| 1964.52002| 1964.52002| 1964.52002|
|2019-07-22|1985.630005|1985.630005|1985.630005|
|2019-07-23| 1994.48999| 1994.48999| 1994.48999|
|2019-07-24|2000.810059|2000.810059|2000.810059|
|2019-07-25|1973.819946|1973.819946|1973.819946|
|2019-07-26|1943.050049|1943.050049|1943.050049|
|2019-07-29|1912.449951|1912.449951|1912.449951|
|2019-07-30|1898.530029|1898.530029|1898.530029|
|2019-07-31|1866.780029|1866.780029|1866.780029|
|2019-08-01|1855.319946|1855.319946|1855.319946|
|2019-08-02| 1823.23999| 1823.23999| 1823.23999|
|2019-08-05|1765.130005|1765.130005|1765.130005|
|2019-08-06|1787.829956|1787.829956|1787.829956|
|2019-08-07|1793.400

In [63]:
final_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- teslaClose: double (nullable = true)
 |-- amazonClose: double (nullable = true)
 |-- googleClose: double (nullable = true)

